In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

import pandas as pd
import hopsworks
from datetime import timedelta
import src.config as config
from src.inference import get_feature_store, fetch_predictions


In [2]:
def fetch_hourly_rides(hours_back=12):
    current_hour = (pd.Timestamp("2025-04-01 00:00:00", tz="UTC") - timedelta(hours=hours_back)).floor('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_NAME,
        version=config.FEATURE_GROUP_VERSION
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour >= current_hour)

    return query.read()

df_actual = fetch_hourly_rides(12)
df_actual.head()


2025-05-03 00:56:32,605 INFO: Initializing external client
2025-05-03 00:56:32,606 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-03 00:56:33,294 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214683
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.58s) 


,pickup_hour,pickup_location_id,rides
0,2025-03-31 08:00:00+00:00,208,0
1,2025-03-31 08:00:00+00:00,84,0
2,2025-03-31 08:00:00+00:00,194,0
3,2025-03-31 08:00:00+00:00,185,1
4,2025-03-31 08:00:00+00:00,60,0


In [3]:
def fetch_recent_predictions(hours_back=12):
    current_hour = (pd.Timestamp("2025-04-01 00:00:00", tz="UTC") - timedelta(hours=hours_back)).floor('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour >= current_hour)

    return query.read()

df_pred = fetch_recent_predictions(12)
df_pred.head()


2025-05-03 00:56:40,558 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-03 00:56:40,561 INFO: Initializing external client
2025-05-03 00:56:40,561 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-03 00:56:41,173 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214683
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.70s) 


,pickup_location_id,predicted_demand,pickup_hour
0,169,1.0,2025-03-31 08:00:00+00:00
1,197,1.0,2025-03-31 08:00:00+00:00
2,239,195.0,2025-03-31 08:00:00+00:00
3,198,0.0,2025-03-31 08:00:00+00:00
4,30,0.0,2025-03-31 08:00:00+00:00


In [4]:
merged_df = pd.merge(
    df_actual,
    df_pred,
    on=['pickup_location_id', 'pickup_hour']
)

merged_df['absolute_error'] = abs(merged_df['predicted_demand'] - merged_df['rides'])
merged_df.head()


,pickup_hour,pickup_location_id,rides,predicted_demand,absolute_error
0,2025-03-31 08:00:00+00:00,208,0,0.0,0.0
1,2025-03-31 08:00:00+00:00,84,0,0.0,0.0
2,2025-03-31 08:00:00+00:00,194,0,0.0,0.0
3,2025-03-31 08:00:00+00:00,185,1,0.0,1.0
4,2025-03-31 08:00:00+00:00,60,0,0.0,0.0


In [5]:
mae_by_hour = (
    merged_df
    .groupby('pickup_hour')['absolute_error']
    .mean()
    .reset_index()
    .rename(columns={'absolute_error': 'MAE'})
)
mae_by_hour.head()


,pickup_hour,MAE
0,2025-03-31 08:00:00+00:00,9.521569
1,2025-03-31 09:00:00+00:00,9.847059
2,2025-03-31 10:00:00+00:00,8.666667
3,2025-03-31 11:00:00+00:00,10.360784
4,2025-03-31 12:00:00+00:00,7.819608


In [6]:
import plotly.express as px

fig = px.line(
    mae_by_hour,
    x='pickup_hour',
    y='MAE',
    title='Mean Absolute Error (MAE) by Pickup Hour',
    labels={'pickup_hour': 'Pickup Hour', 'MAE': 'Mean Absolute Error'},
    markers=True
)

fig.show()


In [7]:
print("🔢 Average MAE across hours:", mae_by_hour["MAE"].mean())


🔢 Average MAE across hours: 8.158348813209495
